In [1]:
import numpy as np
from shapely.geometry import Point
from shapely.ops import cascaded_union
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 994 kB 18.5 MB/s 
     |████████████████████████████████| 6.6 MB 49.7 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 


In [2]:
import folium, folium.plugins
from folium import plugins, FeatureGroup, LayerControl, Map
from folium.plugins import HeatMap, MarkerCluster

In [3]:
import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Preparation

## 1.1 Cluster Grids Generator

In [5]:
## Source Code: https://stackoverflow.com/questions/57000606/create-equally-spaced-coordinates-on-a-california-state-map-created-with-basemap
def generate_grid_in_polygon(spacing, polygon):
    ''' This Function generates evenly spaced points within the given GeoDataFrame.
        The parameter 'spacing' defines the distance between the points in coordinate units. '''

    # Convert the GeoDataFrame to a single polygon
    poly_in = cascaded_union([poly for poly in polygon.geometry])

    # Get the bounds of the polygon
    minx, miny, maxx, maxy = poly_in.bounds    
    
    # Now generate the entire grid
    x_coords = list(np.arange(np.floor(minx), int(np.ceil(maxx)), spacing))
    y_coords = list(np.arange(np.floor(miny), int(np.ceil(maxy)), spacing))
    
    grid = [Point(x) for x in zip(np.meshgrid(x_coords, y_coords)[0].flatten(), np.meshgrid(x_coords, y_coords)[1].flatten())]
    
    # Finally only keep the points within the polygon
    list_of_points = [point for point in grid if point.within(poly_in)]

    # Transform into a normal dataframe with fields: geometry, Longitude, Latitude and ID
    list_of_points_gdf = gpd.GeoDataFrame(geometry=list_of_points)
    grid_points = pd.DataFrame(list_of_points_gdf)
    grid_points['Longitude'] = grid_points.geometry.apply(lambda p: p.x)
    grid_points['Latitude'] = grid_points.geometry.apply(lambda p: p.y)
    grid_points['ID'] = [i for i in range(grid_points.shape[0])]

    return grid_points

In [7]:
df_nybb = gpd.read_file(gpd.datasets.get_path('nybb'))
df_nybb = df_nybb.to_crs(epsg=4326) 
manhattan_map = df_nybb[df_nybb['BoroName']=='Manhattan']

In [ ]:
grid_points = generate_grid_in_polygon(0.002, manhattan_map)

In [ ]:
grid_points

,geometry,Longitude,Latitude,ID
0,POINT (-74.02400 40.68400),-74.024,40.684,0
1,POINT (-74.01200 40.68400),-74.012,40.684,1
2,POINT (-74.02600 40.68600),-74.026,40.686,2
3,POINT (-74.02400 40.68600),-74.024,40.686,3
4,POINT (-74.02200 40.68600),-74.022,40.686,4
...,...,...,...,...
1569,POINT (-73.91200 40.87600),-73.912,40.876,1569
1570,POINT (-73.91000 40.87600),-73.910,40.876,1570
1571,POINT (-73.90800 40.87600),-73.908,40.876,1571
1572,POINT (-73.91200 40.87800),-73.912,40.878,1572


In [6]:
def geo_scatter_plot(df=None, location=[40.767937,-73.982155], tiles="cartodbpositron"):
  '''This function creates a scatter plot of coordinates in the dataset on a base map located by the given location.
  '''
  base_map = folium.Map(location=location, tiles=tiles, control_scale=True, zoom_start=15)
  for each in grid_points.iterrows():
    folium.CircleMarker([each[1]['Latitude'],each[1]['Longitude']], radius=0.001, color='blue',
                        popup=str('(')+str(each[1]['Longitude'])+', '+str(each[1]['Latitude'])+')', fill_color='#FD8A6C').add_to(base_map)
  print('There are '+str(df.shape[0])+' dots on the map.')
  return base_map

In [ ]:
geo_scatter_plot(grid_points)

Output hidden; open in https://colab.research.google.com to view.

- Each blue dot on the map represents a "base cluster"
- There are 1574 "base clusters" in Manhattan area

## 1.2 Data Preparation

In [ ]:
#df_1940 = pd.read_csv('/content/drive/MyDrive/geocoded_clean_data/geocoded_census_1940.csv')
df_1930 = pd.read_csv('/content/drive/MyDrive/geocoded_clean_data/geocoded_census_1930.csv')
#df_1920 = pd.read_csv('/content/drive/MyDrive/geocoded_clean_data/geocoded_census_1920.csv')
#df_1910 = pd.read_csv('/content/drive/MyDrive/geocoded_clean_data/geocoded_census_1910.csv')
#df_1900 = pd.read_csv('/content/drive/MyDrive/geocoded_clean_data/geocoded_census_1900.csv')
df_1880 = pd.read_csv('/content/drive/MyDrive/geocoded_clean_data/geocoded_census_1880.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def df_cleaner(df, year=None):
    """This function cleans geo-coded data frames by separating the field "Coordinates" into two fields: "Latitude" and "longitude"
    """
    df_clean = df.dropna(axis=0, subset = ['Coordinates'])
    df_clean['Latitude'] = [eval(i)[0] for i in df_clean['Coordinates']]
    df_clean['Longitude'] = [eval(i)[1] for i in df_clean['Coordinates']]
    print()
    print('Number of available data in', str(year)+':', df_clean.shape)
    print('Proportion of available data in', str(year)+':', round(df_clean.shape[0]/df.shape[0], 3))
    print()
    return df_clean

In [ ]:
df_1930_clean = df_cleaner(df_1930, 1930)
df_1930_geo = df_1930_clean[['DataId', 'Latitude', 'Longitude']]
del df_1930
df_1880_clean = df_cleaner(df_1880, 1880)
df_1880_geo = df_1880_clean[['DataId', 'Latitude', 'Longitude']]
del df_1880

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



Number of available data in 1930: (1091114, 38)
Proportion of available data in 1930: 0.592



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



Number of available data in 1880: (686736, 31)
Proportion of available data in 1880: 0.577



In [ ]:
df_1930_geo

,DataId,Latitude,Longitude
0,4638830_00956,40.713201,-73.988845
1,4638830_00956,40.713201,-73.988845
2,4638830_00956,40.713201,-73.988845
3,4638830_00956,40.713201,-73.988845
4,4638830_00956,40.713201,-73.988845
...,...,...,...
1844276,4639126_00616,40.823857,-73.942401
1844277,4639126_00615,40.823857,-73.942401
1844278,4639126_00615,40.823857,-73.942401
1844279,4639126_00615,40.823857,-73.942401


# 1.3 Assign Clusters

In [ ]:
def assign_cluster(df, grid_points):
  '''This function assigns each dots in the dataset to its closest cluster based on Euclidean distance. 
  '''
  # 1: round coordinates to three decimal places --> in order to reduce the number of calculations in later steps
  df['Latitude_Mag'] = df['Latitude'].apply(lambda x: round(x, 3))
  df['Longitude_Mag'] = df_1930_clean['Longitude'].apply(lambda x: round(x, 3))
  df_temp = df.groupby(['Latitude_Mag', 'Longitude_Mag'])[['DataId']].count().reset_index()
  df_temp['Cluster'] = [None for i in range(df_temp.shape[0])]

  # 2: calculate the distances of each data point to each cluster centroid;
  #    assign the closest cluster to the data point
  for i in range(df_temp.shape[0]):
    row = df_temp.loc[i,:]
    dist = 100
    c = None
    for j in grid_points.iterrows():
      d = (row['Latitude_Mag'] - j[1]['Latitude'])**2 + (row['Longitude_Mag']-j[1]['Longitude'])**2
      if d < dist:
        dist = d
        c = j[1]['ID']
    df_temp.loc[i, 'Cluster'] = c

  # 3: merge df_temp with df, assign the cluster id to each data point
  df_new = pd.merge(df, df_temp[['Latitude_Mag', 'Longitude_Mag', 'Cluster']], how='left', on=['Latitude_Mag', 'Longitude_Mag'])

  return df_new

In [ ]:
def label_cluster(df, grid_points, s1, s2, s3):
  '''This function labels the degree of size of each cluster based on the number of data points
  '''
  clusters = df.groupby('Cluster')[['DataId']].count().reset_index()
  new_grid_points = pd.merge(grid_points, clusters, how='left', left_on='ID', right_on='Cluster')
  print(new_grid_points['DataId'].describe())

  new_grid_points['Size'] = [None for i in range(grid_points.shape[0])]
  new_grid_points.loc[new_grid_points['DataId']>=s3, 'Size']=5
  new_grid_points.loc[(new_grid_points['DataId']<s3)&(new_grid_points['DataId']>=s2), 'Size']=4
  new_grid_points.loc[(new_grid_points['DataId']<s2)&(new_grid_points['DataId']>=s1), 'Size']=3
  new_grid_points.loc[(new_grid_points['DataId']<s1), 'Size']=2
  new_grid_points.loc[new_grid_points['Cluster'].isnull(), 'Size']=0.01

  new_grid_points.loc[new_grid_points['DataId'].isnull(), 'DataId']=0

  return new_grid_points

## 1.4 Mapping

In [ ]:
def geo_density_describe(new_grid_points, colors=['aliceblue', 'lightblue', 'mediumblue', 'darkblue'], location=[40.767937,-73.982155], 
                         tiles="cartodbpositron", name=None):
  import branca
  import branca.colormap as cm
  colormap = cm.LinearColormap(colors=colors, index=[0.01,2,4,5], vmin=0.01, vmax=5)

  base_map = folium.Map(location=location, tiles=tiles, control_scale=True, zoom_start=13)

  for each in new_grid_points.iterrows():
    folium.CircleMarker([each[1]['Latitude'],each[1]['Longitude']], radius=2**np.log2(each[1]['Size']), 
                        color=colormap(each[1]['Size']), fill_color=colormap(each[1]['Size']), fill_opacity=1,
                        popup="There are "+str(each[1]['DataId'])+" residents in this cluster.").add_to(base_map)
  
  colormap.add_to(base_map)
  # folium.TileLayer("cartodbpositron").add_to(base_map)
  # folium.LayerControl(collapsed=False).add_to(base_map)

  if name is not None:
        base_map.save(r'density_'+name+'.html')
        print('Map saved.')

  return base_map

# 2. Use Cases

## 2.1 1930

In [ ]:
df_1930_geo = assign_cluster(df_1930_geo, grid_points)

In [ ]:
grids_1930 = label_cluster(df_1930_geo, grid_points, s1=100, s2=1000, s3=10000)

count     1105.000000
mean       987.433484
std        973.658852
min          2.000000
25%        266.000000
50%        750.000000
75%       1445.000000
max      14383.000000
Name: DataId, dtype: float64


In [ ]:
df_1930_geo 
# df_1930_geo can be merged with df_1930_clean, using primary key 'DataId'

,DataId,Latitude,Longitude,Latitude_Mag,Longitude_Mag,Cluster
0,4638830_00956,40.713201,-73.988845,40.713,-73.989,92
1,4638830_00956,40.713201,-73.988845,40.713,-73.989,92
2,4638830_00956,40.713201,-73.988845,40.713,-73.989,92
3,4638830_00956,40.713201,-73.988845,40.713,-73.989,92
4,4638830_00956,40.713201,-73.988845,40.713,-73.989,92
...,...,...,...,...,...,...
1091109,4639126_00616,40.823857,-73.942401,40.824,-73.942,1330
1091110,4639126_00615,40.823857,-73.942401,40.824,-73.942,1330
1091111,4639126_00615,40.823857,-73.942401,40.824,-73.942,1330
1091112,4639126_00615,40.823857,-73.942401,40.824,-73.942,1330


In [ ]:
grids_1930
# grids_1930 records the sizes of clusters for the overall 1930 dataset

,geometry,Longitude,Latitude,ID,Cluster,DataId,Size
0,POINT (-74.02400 40.68400),-74.024,40.684,0,NaN,0.0,0.01
1,POINT (-74.01200 40.68400),-74.012,40.684,1,NaN,0.0,0.01
2,POINT (-74.02600 40.68600),-74.026,40.686,2,NaN,0.0,0.01
3,POINT (-74.02400 40.68600),-74.024,40.686,3,NaN,0.0,0.01
4,POINT (-74.02200 40.68600),-74.022,40.686,4,NaN,0.0,0.01
...,...,...,...,...,...,...,...
1569,POINT (-73.91200 40.87600),-73.912,40.876,1569,NaN,0.0,0.01
1570,POINT (-73.91000 40.87600),-73.910,40.876,1570,NaN,0.0,0.01
1571,POINT (-73.90800 40.87600),-73.908,40.876,1571,NaN,0.0,0.01
1572,POINT (-73.91200 40.87800),-73.912,40.878,1572,NaN,0.0,0.01


In [ ]:
m = geo_density_describe(grids_1930, name='blues_1930')
m

Output hidden; open in https://colab.research.google.com to view.

## 2.2 1880

In [ ]:
df_1880_geo = assign_cluster(df_1880_geo, grid_points)
grids_1880 = label_cluster(df_1880_geo, grid_points, s1=100, s2=1000, s3=10000)
geo_density_describe(grids_1880, location=[40.767937,-73.982155], name='blues_1880')

Output hidden; open in https://colab.research.google.com to view.

## 2.3 Comparison, Composite Map

In [ ]:
def geo_density_compare(grids1, grids2, colors=['aliceblue', 'lightblue', 'mediumblue', 'darkblue'], location=None, tiles="cartodbpositron", name=None):
  import branca
  import branca.colormap as cm
  colormap = cm.LinearColormap(colors=colors, index=[0.01,2,4,5], vmin=0.01, vmax=5)

  base_map = folium.Map(location=location, tiles=tiles, control_scale=False, zoom_start=13)

  layer1 = FeatureGroup('Population Density1', overlay=False)
  for each in grids1.iterrows():
    cm = folium.CircleMarker([each[1]['Latitude'],each[1]['Longitude']], radius=2**np.log2(each[1]['Size']), 
                        color=colormap(each[1]['Size']), fill_color=colormap(each[1]['Size']), fill_opacity=1,
                        popup="There are "+str(each[1]['DataId'])+" residents around this cluster center.")
    cm.add_to(layer1)
  layer1.add_to(base_map)

  layer2 = FeatureGroup('Population Density2', overlay=False)
  for each in grids2.iterrows():
    cm = folium.CircleMarker([each[1]['Latitude'],each[1]['Longitude']], radius=2**np.log2(each[1]['Size']), 
                        color=colormap(each[1]['Size']), fill_color=colormap(each[1]['Size']), fill_opacity=1,
                        popup="There are "+str(each[1]['DataId'])+" residents around this cluster center.")
    cm.add_to(layer2)
  layer2.add_to(base_map)

  colormap.add_to(base_map)
  
  folium.LayerControl(collapsed=False).add_to(base_map)
  return base_map

In [ ]:
compare_map1 = geo_density_compare(grids_1930, grids_1880)
compare_map1.save(r'density_blues_1930_and_1880.html')
print('Map saved.')

Map saved.


# Appendix.